## Import modules


In [1]:
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import flatten_fhir_resources, FHIRDataFrame
from spezi_data_pipeline.data_processing.data_processor import FHIRDataProcessor
from spezi_data_pipeline.data_processing.observation_processor import calculate_activity_index
from spezi_data_pipeline.data_exploration.data_explorer import DataExplorer, visualizer_factory, explore_total_records_number
from spezi_data_pipeline.data_export.data_exporter import DataExporter

## Define credential files

In [2]:
# Define your Firebase project ID
project_id = "spezi-data-pipeline"  # Replace with your Firebase project ID
project_id = "stanfordspezitemplateapp"

# Define the service account key file
service_account_key_file = "path_to_service_account_key_file.json"  # Replace with your service account key file
service_account_key_file = "stanfordspezitemplateapp-firebase-adminsdk-thsmt-e0948f0fa1.json"

# Define the collection name where your FHIR observations are stored and the input code if filtering is needed
collection_name = "users"
subcollection_name = "QuestionnaireResponse"

# Define the survey path(s) for creating survey mappings
survey_path = ["SocialSupportQuestionnaire.json"]                

## Initialize the FirebaseFHIRAccess class using your Firebase credentials

In [ ]:
# Initialize the FirebaseFHIRAccess class using your Firebase credentials
firebase_access = FirebaseFHIRAccess(project_id, service_account_key_file)
firebase_access.connect()

fhir_observations = firebase_access.fetch_data(collection_name, subcollection_name)

flattened_fhir_dataframe = flatten_fhir_resources(fhir_observations, survey_path)

In [4]:
flattened_fhir_dataframe.df.head()

,UserId,ResourceId,AuthoredDate,SurveyTitle,SurveyId,QuestionId,QuestionText,AnswerText
0,2VFMFZpWmpNUGUJrDIa7gooJRD12,019A9013-F8D9-4E9F-B17B-6790EFD29D4C,2024-03-27 17:00:24.568549-07:00,Social Support,37DD265A-B9E8-45FA-87E2-50A132AAEA20,dcea2683-9815-4505-b240-e75b502b29ef,How often do you need someone to help you if y...,Most of the time
1,2VFMFZpWmpNUGUJrDIa7gooJRD12,019A9013-F8D9-4E9F-B17B-6790EFD29D4C,2024-03-27 17:00:24.568549-07:00,Social Support,37DD265A-B9E8-45FA-87E2-50A132AAEA20,ce09d701-7b93-4150-defb-51825e05ade9,How often do you need someone to take you to t...,Most of the time
2,2VFMFZpWmpNUGUJrDIa7gooJRD12,019A9013-F8D9-4E9F-B17B-6790EFD29D4C,2024-03-27 17:00:24.568549-07:00,Social Support,37DD265A-B9E8-45FA-87E2-50A132AAEA20,58e97564-5f4d-4d4b-86d5-6429cbbc7a8e,How often do you need someone to prepare your ...,Most of the time
3,2VFMFZpWmpNUGUJrDIa7gooJRD12,019A9013-F8D9-4E9F-B17B-6790EFD29D4C,2024-03-27 17:00:24.568549-07:00,Social Support,37DD265A-B9E8-45FA-87E2-50A132AAEA20,ad161c49-e8a6-4d31-90e8-02b2887a765f,How often do you need someone to help with dai...,Some of the time
4,2VFMFZpWmpNUGUJrDIa7gooJRD12,019A9013-F8D9-4E9F-B17B-6790EFD29D4C,2024-03-27 17:00:24.568549-07:00,Social Support,37DD265A-B9E8-45FA-87E2-50A132AAEA20,ba518851-2843-4bbd-c0f7-5b5692d542e0,What is your age?,65


## Calculate risk score

## Explore Data

## Export data

In [7]:
exporter = DataExporter(flattened_fhir_dataframe)
exporter.export_to_csv("survey_data.csv") 